In [1]:
import lib.xmlreader as xml
import re
import pandas as pd
import os
import gensim
import sys
import numpy as np
import pickle
from lib.utils import tokenize
from collections import Counter

DATA_PATH = '../database/TASS/'

TASS_     = '2018'

In [2]:
train_docs = xml.readXML(DATA_PATH + "TASS"+TASS_+"/task1-Training.xml", [0,1])
test_docs  = xml.readXML(DATA_PATH + "TASS"+TASS_+"/task1-Development.xml", [0,1] )

In [3]:
trn_text   = [t.content  for t in train_docs]
trn_label  = [t.polarity for t in train_docs]

test_text   = [t.content  for t in test_docs]
test_label  = [t.polarity for t in test_docs]

In [4]:
len(trn_text), len(test_text)

(473, 201)

In [5]:
col_names = ['labels','text']

In [6]:
df_train = pd.DataFrame({'text':trn_text, 'labels':trn_label}, columns=col_names)
df_test  = pd.DataFrame({'text':test_text, 'labels':test_label}, columns=col_names)

In [7]:
#CSV_PATH = DATA_PATH + 'csv_data/'

#if not os.path.exists(CSV_PATH):
#    os.makedirs(CSV_PATH)

In [8]:
#df_train.to_csv(DATA_PATH + 'csv_data/train.csv', index=False)
#df_test.to_csv(DATA_PATH  + 'csv_data/test.csv',  index=False)

In [9]:
#df_train = pd.read_csv(DATA_PATH + 'csv_data/train.csv')
#df_test = pd.read_csv(DATA_PATH + 'csv_data/test.csv')

In [10]:
#df_train.head(50)

In [11]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 473 entries, 0 to 472
Data columns (total 2 columns):
labels    473 non-null int64
text      473 non-null object
dtypes: int64(1), object(1)
memory usage: 7.5+ KB


In [12]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201 entries, 0 to 200
Data columns (total 2 columns):
labels    201 non-null int64
text      201 non-null object
dtypes: int64(1), object(1)
memory usage: 3.2+ KB


In [13]:
tokenize(df_train['text'][0], None)

{'original': 'Sin ser fan de Juan Gabriel, siempre supe que era una fuerza de la naturaleza. Hoy escuché "Querida", y me dio una ternura enorme.',
 'words': ['sin',
  'ser',
  'fan',
  'de',
  'juan',
  'gabriel',
  'siempre',
  'supe',
  'que',
  'era',
  'una',
  'fuerza',
  'de',
  'la',
  'naturaleza',
  'hoy',
  'escuche',
  'querida',
  'y',
  'me',
  'dio',
  'una',
  'ternura',
  'enorme'],
 'ratio': 0.05,
 'clean': 'sin ser fan de juan gabriel  siempre supe que era una fuerza de la naturaleza  hoy escuche querida   y me dio una ternura enorme',
 'class': None}

In [14]:
len(df_train['text'])

473

In [15]:
LOF_train = []
LOF_test  = []

for i in range(len(df_train['text'])):
    LOF_train.append(tokenize(df_train['text'][i], None)['words'])
    
for i in range(len(df_test['text'])):
    LOF_test.append(tokenize(df_test['text'][i], None)['words'])

In [16]:
print("Size of List Of Words (Train) : ", len(LOF_train))
print("Size of List Of Words (Test)  : ", len(LOF_test))

Size of List Of Words (Train) :  473
Size of List Of Words (Test)  :  201


In [17]:
#LOF_test[0]

In [18]:
cnt = Counter(word for doc in LOF_train + LOF_test for word in doc )

In [19]:
cnt['felicidad'], len(cnt)

(3, 3067)

In [20]:
cnt.most_common()[-1]

('gigante', 1)

In [21]:
cnt.most_common()[:10]

[('que', 426),
 ('de', 343),
 ('y', 277),
 ('a', 233),
 ('la', 226),
 ('no', 212),
 ('es', 196),
 ('el', 195),
 ('me', 194),
 ('en', 170)]

In [22]:
import collections

max_vocab = len(cnt)
min_freq = 3
#max_vocab = len(cnt)
#max_vocab = 4000
#min_freq  = 0


itos = [o for o,c in cnt.most_common(max_vocab) if c >= min_freq]

itos.insert(0, '_pad_')
itos.insert(0, '_unk_')

stoi = collections.defaultdict(lambda:0, {v:k for k,v in enumerate(itos)})
len(itos)

480

In [23]:
len(stoi)

480

In [60]:
train_lm = np.array([[stoi[o] for o in p] for p in LOF_train])
test_lm  = np.array([[stoi[o] for o in p] for p in LOF_test])

In [61]:
SAVE_PATH = '../database/ulmfit/'
np.save(SAVE_PATH + "tmp/train_labels_"+TASS_+".npy",trn_label)
np.save(SAVE_PATH + "tmp/test_labels_"+TASS_+".npy",test_label)

In [62]:
np.save(SAVE_PATH + "tmp/train_lm_"+TASS_+".npy",train_lm)
np.save(SAVE_PATH + "tmp/test_lm_"+TASS_+".npy",test_lm)
pickle.dump(itos, open(SAVE_PATH + "tmp/itos_"+TASS_+"_freq_"+str(min_freq)+'.pkl', 'wb'))